In [99]:
import galois
import sys
from py_ecc.bls12_381 import G1, G2, G12, add, multiply, curve_order, pairing, eq, neg, Z1
import random
sys.path.append('../hw_1')
from pedersen_commitment import generate_n_random_points, pedersen_commitment

In [100]:
GF = galois.GF(curve_order)

In [101]:
G = generate_n_random_points("goblin-plonk", 1)[0]
G_vec = generate_n_random_points("afkjhweiuljbfwea", 3)
H = generate_n_random_points("goblin", 1)[0]
H_vec = generate_n_random_points("cwea", 3)
B = generate_n_random_points("whatsup", 1)[0]

In [ ]:
class VectorPolynomial:
    def __init__(self, coefficients):
        if coefficients:
            dim = len(coefficients[0])
            if not all(len(v) == dim for v in coefficients):
                raise ValueError("All coefficient vectors must have the same dimension")
        
        self.coefficients = coefficients
        self.degree = len(coefficients) - 1 if coefficients else -1
        
    def evaluate(self, x):
        if not self.coefficients:
            return []
            
        result = [GF(0)] * len(self.coefficients[0])
        x_power = 1
        
        for coeff_vector in self.coefficients:
            term = [x_power * c for c in coeff_vector]
            result = [r + t for r, t in zip(result, term)]
            x_power *= x
            
        return result
    
    def __str__(self):
        if not self.coefficients:
            return "0"
            
        terms = []
        for power, vector in enumerate(self.coefficients):
            if power == 0:
                terms.append(f"{vector}")
            elif power == 1:
                terms.append(f"{vector}x")
            else:
                terms.append(f"{vector}x^{power}")
                
        return " + ".join(terms)


In [103]:
def calculate_inner_product(a, b):
  if len(a) != len(b):
    raise ValueError("Lists must be of the same length")
  result = GF(0)
  for index, val in enumerate(a):
    result += val * b[index]
  return result

In [104]:
def calculate_inner_product_with_ec(a, G):
  result = Z1
  for index, val in enumerate(a):
    result = add(result, multiply(G[index], int(val)))
  
  return result

In [ ]:
def generate_inner_product_commitment(a, b):
  v = calculate_inner_product(a, b)
  s_l = [GF(random.randint(1, 1000)) for _ in range(len(a))]
  s_r = [GF(random.randint(1, 1000)) for _ in range(len(b))]

  l = VectorPolynomial([a, s_l])
  r = VectorPolynomial([b, s_r])

  t_coeffs = [calculate_inner_product(s_r, s_l), calculate_inner_product(a, s_r) + calculate_inner_product(b, s_l), v]
  t = galois.Poly(t_coeffs, GF)
  alpha = GF(random.randint(0, 10000))

  A = add(add(calculate_inner_product_with_ec(a, G_vec), calculate_inner_product_with_ec(b, H_vec)), multiply(B, int(alpha)))

  beta = GF(random.randint(1, 10000))
  S = add(add(calculate_inner_product_with_ec(s_l, G_vec), calculate_inner_product_with_ec(s_r, H_vec)), multiply(B, int(beta)))

  gamma = GF(random.randint(0, 10000))
  V = add(multiply(G, int(v)), multiply(B, int(gamma)))

  tau_1 = GF(random.randint(1, 1000000))
  t_1_scalar = calculate_inner_product(a, s_r) + calculate_inner_product(b, s_l)
  T_1 = add(multiply(G, int(t_1_scalar)), multiply(B, int(tau_1)))
  
  tau_2 = GF(random.randint(0, 100000))
  T_2 = add(multiply(G, int(calculate_inner_product(s_l, s_r))), multiply(B, int(tau_2)))

  return [(A, S, V, T_1, T_2), (l, r, t), (alpha, beta, gamma, tau_1, tau_2)]
  


In [106]:
a = [GF(20), GF(14), GF(40)]
b = [GF(34), GF(45), GF(60)]
[(A, S, V, T_1, T_2), (l, r, t), (alpha, beta, gamma, tau_1, tau_2)] = generate_inner_product_commitment(a, b)
u = GF(random.randint(1, 10000))



3710


In [107]:
def generate_inner_product_proof(u):
  l_u = l.evaluate(u)
  r_u = r.evaluate(u)
  t_u = t(u)

  pi_lr = alpha + beta * u
  pi_t = gamma + tau_1 * u + tau_2 * u * u

  return (l_u, r_u, t_u, pi_lr, pi_t)

In [108]:
(l_u, r_u, t_u, pi_lr, pi_t) = generate_inner_product_proof(u)

In [111]:
def verify_proof(commitments, proofs, u):
	(A, S, V, T_1, T_2) = commitments
	(l_u, r_u, t_u, pi_lr, pi_t) = proofs
	
	if t_u != calculate_inner_product(l_u, r_u):
		raise "Invalid Proof"
	
	eqn_2_lhs = add(A, multiply(S, int(u)))
	eqn_2_rhs = add(add(calculate_inner_product_with_ec(l_u, G_vec), calculate_inner_product_with_ec(r_u, H_vec)), multiply(B, int(pi_lr)))
	
	if eqn_2_lhs != eqn_2_rhs:
		raise "Invalid Proof"

	eqn_3_lhs = add(multiply(G, int(t_u)), multiply(B, int(pi_t)))
	eqn_3_rhs = add(V, add(multiply(T_1, int(u)), multiply(T_2, int(u * u))))

	if eqn_3_lhs != eqn_3_rhs:
		raise "Invalid Proof"
	
	return True




In [112]:
isVerified = verify_proof((A, S, V, T_1, T_2), (l_u, r_u, t_u, pi_lr, pi_t), u )

print(isVerified)

True
